In [ ]:
import subprocess
from os.path import join
from time import sleep
from chainrunner import Chain, Seq, generate_keys

In [ ]:
# Path to working directory, where chainspec, logs and nodes' dbs are written:
workdir = '/tmp/workdir'
# Path to the pre-update aleph-node binary:
oldbin = join(workdir,'aleph-node-old')
# Path to the post-update aleph-node binary:
newbin = join(workdir,'aleph-node-new')
# Path to the post-update compiled runtime:
runtime = join(workdir,'aleph_runtime.compact.wasm')
# Path to the send-runtime binary (which lives in aleph-node/local-tests/send-runtime):
send_runtime = 'send-runtime/target/release/send_runtime'

In [ ]:
def query_runtime_version(chain):
    print('Current version:')
    for i,node in enumerate(chain):
        sys = node.rpc('system_version').result
        rt = node.rpc('state_getRuntimeVersion').result['specVersion']
        print(f'  Node {i}: system: {sys}  runtime: {rt}')

In [ ]:
def check_highest(chain):
    results = [node.highest_block() for node in chain]
    highest, finalized = zip(*results)
    print('Blocks seen by nodes:')
    print('  Highest:   ', *highest)
    print('  Finalized: ', *finalized)

In [ ]:
phrases = ['//Cartman','//Stan','//Kyle','//Kenny']
keys = generate_keys(newbin, phrases)

In [ ]:
chain = Chain(workdir)
chain.bootstrap(oldbin, keys.values(), sudo_account_id=keys[phrases[0]], chain_type='local', millisecs_per_block=2000, session_period=40)

In [ ]:
chain.set_flags('validator', port=Seq(30334), ws_port=Seq(9944), rpc_port=Seq(9933), unit_creation_delay=200, execution='Native')

In [ ]:
chain.start('old')

In [ ]:
check_highest(chain)
query_runtime_version(chain)

In [ ]:
chain[3].stop()
chain[3].purge()
chain[3].binary = newbin
chain[3].start('new3')

In [ ]:
check_highest(chain)
query_runtime_version(chain)

In [ ]:
subprocess.check_call([send_runtime, '--url', 'localhost:9945', '--sudo-phrase', phrases[0], runtime])

In [ ]:
check_highest(chain)
query_runtime_version(chain)

In [ ]:
chain.stop(nodes=[0,1,2])
chain.set_binary(newbin, nodes=[0,1,2])
chain.start('new', nodes=[0,1,2])

In [ ]:
check_highest(chain)
query_runtime_version(chain)

In [ ]:
chain.stop()

In [ ]:
hf = min(node.highest_block()[1] for node in chain)
print(f'Sanity check: the highest finalized block is {hf}. Comparing exported states after that block:')
if (chain[0].state(hf) == chain[1].state(hf) == chain[2].state(hf) == chain[3].state(hf)):
    print("The same :)")
else:
    print("DIFFERENT!")